In [ ]:
%pip install -q networkx==3.2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import networkx as nx
import pandas as pd
import torch.nn.functional as F
import torch.optim as optim
import random
from evaluation_functions import KFold
from evaluation import evaluate_all

In [ ]:
data = KFold(embeddings='Random CV/Embeddings/lr_embeddings')

In [ ]:
# Set a fixed random seed for reproducibility across multiple libraries
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Check for CUDA (GPU support) and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.set_default_device(device)
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

In [ ]:
def weight_variable_glorot(output_dim):
    input_dim = output_dim
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = np.random.uniform(-init_range, init_range, (input_dim, output_dim))
    return initial


def pad_HR_adj(label, split):
    padded_label = torch.nn.functional.pad(label, ((split, split, split, split)), mode="constant")
    padded_label.fill_diagonal_(0)
    return padded_label


def normalize_adj_torch(mx):
    rowsum = mx.sum(1)
    r_inv_sqrt = torch.pow(rowsum, -0.5).flatten()
    r_inv_sqrt[torch.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = torch.diag(r_inv_sqrt)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    mx = torch.transpose(mx, 0, 1)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    return mx


def unpad(data, split):
    idx_0 = data.shape[0]-split
    idx_1 = data.shape[1]-split
    train = data[split:idx_0, split:idx_1]
    return train


# For Drop-GNN Integration to Forward pass for AGSRNet
def apply_dropout(A, dropout_prob=0.02):
    dropout_mask = torch.rand(A.shape) > dropout_prob
    dropout_mask = dropout_mask.to(device)
    A_dropped = A * dropout_mask.float()
    return A_dropped

In [ ]:
class GCLayer(nn.Module):
    """
    Graph convolutional layer with attention
    """

    def __init__(self, in_features, out_features, act=F.relu):
        super(GCLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.act = act
        self.weight = torch.nn.Parameter(torch.FloatTensor(in_features, out_features).to(device))
        self.bias = nn.Parameter(torch.zeros(out_features).to(device))
        self.phi = nn.Parameter(torch.FloatTensor(2 * out_features, 1).to(device))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)
        torch.nn.init.xavier_uniform_(self.phi)

    def forward(self, A, X):
        # 1. Apply linear transformation and add bias
        l = torch.mm(X, self.weight)
        N = l.size(0)

        # 2. Compute the attention coefficients
        a_input = torch.cat([l.repeat(1, N).view(N * N, -1), l.repeat(N, 1)], dim=1).view(N, N, -1).to(device)
        S = torch.matmul(a_input, self.phi).view(N, N)

        # 3. Compute mask based on adjacency matrix
        I = torch.eye(N, device=S.device)
        mask = (A + I).bool()
        masked_S = torch.where(mask, S, torch.tensor(float('-inf'), device=S.device))

        # 4. Apply softmax to compute attention weights
        attention = F.softmax(masked_S, dim=1)

        # 5. Aggregate features based on attention weights
        h = torch.matmul(attention, l)

        return self.act(h)

In [ ]:
class GSRLayer(nn.Module):

    def __init__(self, out_dim):
        super(GSRLayer, self).__init__()
        self.weights = torch.nn.Parameter(
            torch.from_numpy(weight_variable_glorot(out_dim)).type(torch.FloatTensor).to(device))

    def forward(self, A, X):
        with torch.autograd.set_detect_anomaly(True):
            lr = A.to(device)
            f = X.to(device)

            lr_dim = lr.shape[0]
            eye_mat = torch.eye(lr_dim).type(torch.FloatTensor).to(device)
            s_d = torch.cat((eye_mat, eye_mat), dim=0)

            eig_val_lr, U_lr = torch.linalg.eigh(lr, UPLO='U')

            a = torch.matmul(self.weights, s_d)
            b = torch.matmul(a, torch.t(U_lr))
            f_d = torch.matmul(b, f)
            f_d = torch.abs(f_d)
            f_d = f_d.fill_diagonal_(0)
            A = f_d

            X = torch.mm(A, A.t())
            X = (X + X.t()) / 2
            X = X.fill_diagonal_(0)
        return A, torch.abs(X)

In [ ]:
class GraphUnpool(nn.Module):

    def __init__(self):
        super(GraphUnpool, self).__init__()

    def forward(self, A, X, idx):
        new_X = torch.zeros([A.shape[0], X.shape[1]])
        new_X[idx] = X
        return A, new_X


class GraphPool(nn.Module):

    def __init__(self, k, in_dim):
        super(GraphPool, self).__init__()
        self.k = k
        self.proj = nn.Linear(in_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, A, X):
        scores = self.proj(X)
        scores = torch.squeeze(scores)
        scores = self.sigmoid(scores / 100)
        num_nodes = A.shape[0]
        values, idx = torch.topk(scores, int(self.k * num_nodes))
        new_X = X[idx, :]
        values = torch.unsqueeze(values, -1)
        new_X = torch.mul(new_X, values)
        A = A[idx, :]
        A = A[:, idx]
        return A, new_X, idx


class GraphUnet(nn.Module):

    def __init__(self, ks, in_dim, out_dim, dim=320):
        super(GraphUnet, self).__init__()
        self.ks = ks

        self.start_gcn = GCLayer(in_dim, dim)
        self.bottom_gcn = GCLayer(dim, dim)
        self.end_gcn = GCLayer(2 * dim, out_dim)
        self.down_gcns = []
        self.up_gcns = []
        self.pools = []
        self.unpools = []
        self.l_n = len(ks)
        for i in range(self.l_n):
            self.down_gcns.append(GCLayer(dim, dim))
            self.up_gcns.append(GCLayer(dim, dim))
            self.pools.append(GraphPool(ks[i], dim))
            self.unpools.append(GraphUnpool())

    def forward(self, A, X):
        adj_ms = []
        indices_list = []
        down_outs = []
        X = self.start_gcn(A, X)
        start_gcn_outs = X
        org_X = X
        for i in range(self.l_n):

            X = self.down_gcns[i](A, X)
            adj_ms.append(A)
            down_outs.append(X)
            A, X, idx = self.pools[i](A, X)
            indices_list.append(idx)

        X = self.bottom_gcn(A, X)
        for i in range(self.l_n):
            up_idx = self.l_n - i - 1

            A, idx = adj_ms[up_idx], indices_list[up_idx]
            A, X = self.unpools[i](A, X, idx)
            X = self.up_gcns[i](A, X)
            X = X.add(down_outs[up_idx])
        X = torch.cat([X, org_X], 1)

        X = self.end_gcn(A, X)

        return X, start_gcn_outs

In [ ]:
class AGSRNet(nn.Module):

    def __init__(self, ks, args):
        super(AGSRNet, self).__init__()

        self.lr_dim = args.lr_dim
        self.hr_dim = args.hr_dim
        self.hidden_dim = args.hidden_dim
        self.conv1 = GCLayer(self.lr_dim, self.hidden_dim)
        self.net = GraphUnet(ks, self.hidden_dim, self.hr_dim)
        self.gsr_layer = GSRLayer(self.hr_dim)

    def forward(self, lr, embeddings, num_runs=3):
        with torch.autograd.set_detect_anomaly(True):

            A = normalize_adj_torch(lr).type(torch.FloatTensor).to(device)

            X = embeddings
            X = self.conv1(A, X)

            # DropGNN Contribution: Store the embeddings from all runs
            all_net_outs, all_start_gcn_outs, all_outputs, all_Z = [], [], [], []

            for _ in range(num_runs):
                A_dropout = apply_dropout(A)

                net_outs, start_gcn_outs = self.net(A_dropout, X)
                outputs, Z = self.gsr_layer(A_dropout, net_outs)

                # DropGNN Contribution: Store intermediate results for aggregation
                all_net_outs.append(net_outs.unsqueeze(0))
                all_start_gcn_outs.append(start_gcn_outs.unsqueeze(0))
                all_outputs.append(outputs.unsqueeze(0))
                all_Z.append(Z.unsqueeze(0))

            # DropGNN Contribution: Aggregate
            net_outs = torch.mean(torch.cat(all_net_outs, dim=0), dim=0)
            start_gcn_outs = torch.mean(torch.cat(all_start_gcn_outs, dim=0), dim=0)
            outputs = torch.mean(torch.cat(all_outputs, dim=0), dim=0)
            Z = torch.mean(torch.cat(all_Z, dim=0), dim=0)

            Z = (Z + Z.t()) / 2
            Z.fill_diagonal_(0)

        return torch.abs(Z), net_outs, start_gcn_outs, outputs

In [ ]:
class Dense(nn.Module):
    def __init__(self, n1, n2, args):
        super(Dense, self).__init__()
        self.weights = torch.nn.Parameter(torch.FloatTensor(n1, n2).to(device))
        nn.init.normal_(self.weights, mean=args.mean_dense, std=args.std_dense)

    def forward(self, X):
        out = torch.mm(X, self.weights)
        return out


class Discriminator(nn.Module):
    def __init__(self, args):
        super(Discriminator, self).__init__()
        self.dense_1 = Dense(args.hr_dim, args.hr_dim, args)
        self.relu_1 = nn.ReLU(inplace=False)
        self.dense_2 = Dense(args.hr_dim, args.hr_dim, args)
        self.relu_2 = nn.ReLU(inplace=False)
        self.dense_3 = Dense(args.hr_dim, 1, args)
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs):
        dc_den1 = self.relu_1(self.dense_1(inputs))
        dc_den2 = self.relu_2(self.dense_2(dc_den1))
        output = dc_den2
        output = self.dense_3(dc_den2)
        output = self.sigmoid(output)
        return torch.abs(output)


def gaussian_noise_layer(input_layer, args):
    z = torch.empty_like(input_layer)
    noise = z.normal_(mean=args.mean_gaussian, std=args.std_gaussian)
    z = torch.abs(input_layer + noise)
    z = (z + z.t())/2
    z = z.fill_diagonal_(0)
    return z

In [ ]:
criterion = nn.MSELoss()

def train_for_epoch(model, subjects_adj, subjects_labels, val_adj, val_labels, embeddings, val_embeddings, args, min_epochs=100, early_stopping=10):

    bce_loss = nn.BCELoss()
    netD = Discriminator(args).to(device)

    optimizerG = optim.Adam(model.parameters(), lr=args.lr)
    optimizerD = optim.Adam(netD.parameters(), lr=args.lr)

    all_epochs_training_loss = []
    all_epochs_training_error = []
    all_epochs_val_loss = []
    all_epochs_val_error = []

    best_val_error = float('inf')
    best_epoch = 0

    epoch = 0
    epoch_without_improvement = 0

    while epoch < min_epochs or epoch_without_improvement < early_stopping:
        epoch += 1
        with torch.autograd.set_detect_anomaly(True):
            epoch_training_loss = []
            epoch_training_error = []
            epoch_val_loss = []
            epoch_val_error = []
            model.train()
            for lr, hr, x in zip(subjects_adj, subjects_labels, embeddings):
                optimizerD.zero_grad()
                optimizerG.zero_grad()

                lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
                hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)
                x = torch.from_numpy(x).type(torch.FloatTensor).to(device)

                model_outputs, net_outs, start_gcn_outs, layer_outs = model(lr, x)

                net_outs = net_outs.to(device)
                start_gcn_outs = start_gcn_outs.to(device)
                model_outputs = model_outputs.to(device)

                padded_hr = pad_HR_adj(hr, args.padding)
                eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

                mse_loss1 = args.lmbda * criterion(net_outs, start_gcn_outs)
                mse_loss2 = criterion(model.gsr_layer.weights, U_hr)
                mse_loss3 = criterion(model_outputs, padded_hr)
                mse_loss = mse_loss1 + mse_loss2 + mse_loss3

                error = criterion(model_outputs, padded_hr)
                real_data = model_outputs.detach()
                fake_data = gaussian_noise_layer(padded_hr, args)

                d_real = netD(real_data)
                d_fake = netD(fake_data)

                dc_loss_real = bce_loss(d_real, torch.ones(args.hr_dim, 1))
                dc_loss_fake = bce_loss(d_fake, torch.zeros(args.hr_dim, 1))
                dc_loss = dc_loss_real + dc_loss_fake

                dc_loss.backward()
                optimizerD.step()

                d_fake = netD(gaussian_noise_layer(padded_hr, args))

                gen_loss = bce_loss(d_fake, torch.ones(args.hr_dim, 1))
                generator_loss = gen_loss + mse_loss
                generator_loss.backward()
                optimizerG.step()

                epoch_training_loss.append(generator_loss.item())
                epoch_training_error.append(error.item())

            model.eval()
            with torch.no_grad():
                for lr, hr, x in zip(val_adj, val_labels, val_embeddings):

                    lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
                    hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)
                    x = torch.from_numpy(x).type(torch.FloatTensor).to(device)

                    model_outputs, net_outs, start_gcn_outs, layer_outs = model(lr, x)

                    net_outs = net_outs.to(device)
                    start_gcn_outs = start_gcn_outs.to(device)
                    model_outputs = model_outputs.to(device)

                    padded_hr = pad_HR_adj(hr, args.padding)
                    eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

                    mse_loss1 = args.lmbda * criterion(net_outs, start_gcn_outs)
                    mse_loss2 = criterion(model.gsr_layer.weights, U_hr)
                    mse_loss3 = criterion(model_outputs, padded_hr)
                    mse_loss = mse_loss1 + mse_loss2 + mse_loss3

                    error = criterion(model_outputs, padded_hr)
                    real_data = model_outputs.detach()
                    fake_data = gaussian_noise_layer(padded_hr, args)

                    d_real = netD(real_data)
                    d_fake = netD(fake_data)

                    dc_loss_real = bce_loss(d_real, torch.ones(args.hr_dim, 1))
                    dc_loss_fake = bce_loss(d_fake, torch.zeros(args.hr_dim, 1))
                    dc_loss = dc_loss_real + dc_loss_fake

                    d_fake = netD(gaussian_noise_layer(padded_hr, args))

                    gen_loss = bce_loss(d_fake, torch.ones(args.hr_dim, 1))
                    generator_loss = gen_loss + mse_loss
                    
                    epoch_val_loss.append(generator_loss.item())
                    epoch_val_error.append(error.item())

            print("Epoch: ", epoch, "Loss: ", np.mean(epoch_training_loss),
                    "Error: ", np.mean(epoch_training_error) * 100, "%", "Val Loss: ", np.mean(epoch_val_loss),
                    "Val Error: ", np.mean(epoch_val_error) * 100, "%")
            all_epochs_training_loss.append(np.mean(epoch_training_loss))
            all_epochs_training_error.append(np.mean(epoch_training_error))
            all_epochs_val_loss.append(np.mean(epoch_val_loss))
            all_epochs_val_error.append(np.mean(epoch_val_error))

            epoch_without_improvement += 1

            if np.mean(epoch_val_error) < best_val_error:
                best_val_error = np.mean(epoch_val_error)
                best_epoch = epoch
                epoch_without_improvement = 0

    return best_epoch, all_epochs_training_loss, all_epochs_training_error, all_epochs_val_loss, all_epochs_val_error


def train(model, subjects_adj, subjects_labels, embeddings, args, num_epochs):

    bce_loss = nn.BCELoss()
    netD = Discriminator(args).to(device)

    optimizerG = optim.Adam(model.parameters(), lr=args.lr)
    optimizerD = optim.Adam(netD.parameters(), lr=args.lr)

    all_epochs_loss = []

    for epoch in range(num_epochs):
        with torch.autograd.set_detect_anomaly(True):
            epoch_loss = []
            epoch_error = []
            model.train()
            for lr, hr, x in zip(subjects_adj, subjects_labels, embeddings):
                optimizerD.zero_grad()
                optimizerG.zero_grad()

                lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
                hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)
                x = torch.from_numpy(x).type(torch.FloatTensor).to(device)

                model_outputs, net_outs, start_gcn_outs, layer_outs = model(lr, x)

                net_outs = net_outs.to(device)
                start_gcn_outs = start_gcn_outs.to(device)
                model_outputs = model_outputs.to(device)

                padded_hr = pad_HR_adj(hr, args.padding)
                eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

                mse_loss1 = args.lmbda * criterion(net_outs, start_gcn_outs)
                mse_loss2 = criterion(model.gsr_layer.weights, U_hr)
                mse_loss3 = criterion(model_outputs, padded_hr)
                mse_loss = mse_loss1 + mse_loss2 + mse_loss3

                error = criterion(model_outputs, padded_hr)
                real_data = model_outputs.detach()
                fake_data = gaussian_noise_layer(padded_hr, args)

                d_real = netD(real_data)
                d_fake = netD(fake_data)

                dc_loss_real = bce_loss(d_real, torch.ones(args.hr_dim, 1))
                dc_loss_fake = bce_loss(d_fake, torch.zeros(args.hr_dim, 1))
                dc_loss = dc_loss_real + dc_loss_fake

                dc_loss.backward()
                optimizerD.step()

                d_fake = netD(gaussian_noise_layer(padded_hr, args))

                gen_loss = bce_loss(d_fake, torch.ones(args.hr_dim, 1))
                generator_loss = gen_loss + mse_loss
                generator_loss.backward()
                optimizerG.step()

                epoch_loss.append(generator_loss.item())
                epoch_error.append(error.item())

            print("Epoch: ", epoch, "Loss: ", np.mean(epoch_loss),
                    "Error: ", np.mean(epoch_error) * 100, "%")
            all_epochs_loss.append(np.mean(epoch_loss))

# Training and Evaluation

In [ ]:
matrix_size_lr = 160
matrix_size_hr = 268

data.preprocessing(matrix_size_lr, matrix_size_hr)

In [ ]:
def generate_predictions(model, test_adj, test_labels, embeddings, args, vectorize=True, file=None):
    model.eval()
    preds_list = []

    with torch.no_grad():
        for lr, x in zip(test_adj, embeddings):
            all_zeros_lr = not np.any(lr)
            if all_zeros_lr == False:
                lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
                x = torch.from_numpy(x).type(torch.FloatTensor).to(device)
                preds, a, b, c = model(lr, x)

                preds_square = preds.view(args.hr_dim, args.hr_dim).cpu().numpy()
                truncated_preds = preds_square[args.padding:-args.padding, args.padding:-args.padding]

                if vectorize:
                    final_preds = data.mv.vectorize(truncated_preds)
                else:
                  final_preds = truncated_preds

                preds_list.append(final_preds)

    all_preds = np.array(preds_list)

    _test_labels = np.array([np.array([np.array(x) for x in p]) for p in test_labels])

    metrics = evaluate_all(
            _test_labels, all_preds, output_path=file
        )

    return all_preds, None

In [ ]:
class Args:
    lr = 0.0001
    lmbda = 0.1
    lr_dim = 160
    hr_dim = 320
    hidden_dim = 320
    padding = 26
    mean_dense = 0.0
    std_dense = 0.01
    mean_gaussian = 0.0
    std_gaussian = 0.1

args = Args()

ks = [0.9, 0.7, 0.6, 0.5]

In [ ]:
def save_losses(test_index, training_losses, training_error, validation_losses=None, validation_error=None):
    fig = plt.figure()
    plt.plot(training_losses, label='Training loss')
    plt.plot(training_error, label='Training error')
    if validation_losses is not None:
        plt.plot(validation_losses, label='Validation loss')
        plt.plot(validation_error, label='Validation error')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Fold ' + str(test_index))
    plt.savefig(f'./evaluation/Random CV/loss_fold_{test_index}_validation.png' if validation_losses is not None else f'./evaluation/Random CV/loss_fold_{test_index}_training.png')
    plt.close(fig)

In [ ]:
#lr_data_adjacency_matrices = np.array(lr_data_adjacency_matrices)
#hr_data_adjacency_matrices = np.array(hr_data_adjacency_matrices)

def cross_validation_with_embeddings(k=3):
    fold_predictions = []
    fold_ground_truths = []

    for test_index in range(3):
        print(f"Fold {test_index+1}:")
        #reset model/ initialise
        model = AGSRNet(ks, args).to(device)
        train_adj, train_ground_truth, dev_adj, dev_ground_truth, train_embeddings, dev_embeddings = data.obtain_folds(test_index, with_validation=True, return_embeddings=True)
        best_epoch, all_epochs_training_loss, all_epochs_training_error, all_epochs_val_loss, all_epochs_val_error = train_for_epoch(model, train_adj, train_ground_truth, dev_adj, dev_ground_truth, train_embeddings, dev_embeddings, args)
        save_losses(test_index+1, all_epochs_training_loss, all_epochs_training_error, all_epochs_val_loss, all_epochs_val_error)
        print(f"Best epoch for fold {test_index+1}: {best_epoch}")
        train_adj, train_ground_truth, dev_adj, dev_ground_truth, train_embeddings, dev_embeddings = data.obtain_folds(test_index, with_validation=False, return_embeddings=True)
        model = AGSRNet(ks, args).to(device)
        train(model, train_adj, train_ground_truth, train_embeddings, args, best_epoch)
        fold_pred, _ = generate_predictions(model, dev_adj, dev_ground_truth, dev_embeddings, args, vectorize=False, file=f'./evaluation/Random CV/metrics.csv')
        # Post-process predictions
        fold_pred = np.clip(fold_pred, 0, 1)
        # Save predictions in CSV
        vec_fold_pred = data.mv.vectorize(fold_pred)
        flat_fold_pred = vec_fold_pred.flatten()
        ids = np.arange(1, len(flat_fold_pred) + 1)
        submission_df = pd.DataFrame({
            'ID': ids,
            'Predicted': flat_fold_pred
        })
        submission_df.to_csv(f'./evaluation/Random CV/predictions_fold_{test_index+1}.csv', index=False)
        # Store predictions and ground truths
        fold_predictions.append(fold_pred)
        fold_ground_truths.append(dev_ground_truth)

    return fold_predictions, fold_ground_truths

# Perform cross-validation
cv_preds, cv_gts = cross_validation_with_embeddings(k=3)